# Initialization

### Installation

Version used for run notebook

* gdown: 5.1.0
* tqdm: 4.66.4
* torchmetrics: 1.4.0.post0
* scikit-learn: Version: 1.2.2*
* pykan: 0.1.1

In [1]:
%%capture
!pip install gdown
!pip install tqdm
!pip install torchvision --quiet
!pip install pykan==0.1.1
!pip install torchmetrics
!pip install scikit-learn

### Import of libraries

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torchvision.transforms import ToPILImage
from torchvision.io import read_image
from torch.utils.data import random_split, DataLoader, Dataset

from torchmetrics.classification import BinaryAccuracy
from torchmetrics.classification import BinaryPrecision
from torchmetrics.classification import BinaryRecall
from torchmetrics.classification import BinaryAUROC
from torchmetrics.classification import BinaryF1Score

from kan import MultKAN as KAN

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix

import os
import gdown
import zipfile
import shutil
import gc
from tqdm.notebook import tqdm
import time
from google.colab import drive
import json

import random


### GLOBAL DEFINITIONS

Global variables where there are stored hyperparameters for the training and link, paths for the dataset.

In [3]:
global_var = {
    # Dataset
    'link_download_prefix' : 'https://drive.google.com/uc?export=download&id=YOUR_FILE_ID', # Generic link to download from Drive the file with id "YOUR_FILE_ID"
    "link_dataset": "https://drive.google.com/file/d/10un_W7teXQy9LOB1uqd0e6VJ46UFNRza/view?usp=drive_link0", # The ID is "10un_W7teXQy9LOB1uqd0e6VJ46UFNRza" of the file
    "path_download_dataset": "/content/dataset.zip", # Path of the runtime directory of Colab with the zipped dataset
    "extract_directory": "/content", # Directory where we want to extract the zipped dataset
    "dataset_name": "chest_xray", # Name of the dataset
    "dataset_name_resized": "chest_xray_resized", # Name of the dataset resized
    "dataset_name_resized_augmented": "chest_xray_resized_augmented", # Name of the dataset resized
    "dataset_name_gaussian": "chest_xray_gaussian",
    "dataset_name_gaussian_he": "chest_xray_gaussian_he",
    "dataset_name_gaussian_clahe": "chest_xray_gaussian_clahe",
    "sub_folders_ttv": ['train', 'test', 'val'], # at least must exist one between {'train', 'test', 'val'}

    # Preprocessing parameters
    "resizing_dim": (256,256),
    "smoothing_value": 3,
    "augment_performed": False,  # if want to perform augmentation
    "augmentation_train_dimension": 1125,
    "augmentation_test_dimension": 156,


    # Subset parameters
    "use_subset": True, # if want to train on a subset
    "subset_suffix" : "_subset",
    "split_value": [0.7, 0.2, 0.1], # train, test, val splitting values
    "subset_dim_percentage": 0.1, # from 0.1 to 1

    # Train
    'batch_size': 48,
    "train_epochs": 20,
    'early_stop_threshold': 1e-4,
    'early_stop_patience': 2,

}

###REPRODUCIBILITY

This part sets the seed for various random number generators of torch, numpy and random libraries, to ensure that the results are the same each time the code is run.

CuBLAS is a GPU-accelerated library for dense linear algebra operations, such as matrix multiplications, that is part of the CUDA toolkit provided by NVIDIA: we have set the environment variable of the workspace of CuBLAS to "_:4096:8_" to ensure deterministic behaviour each run.

1. **4096** => Size in byte of the workspace used by CuBLAS for the operation
2. **8** => Parameter that ensure to use deterministic algorithm

To ensure reproducibility we have also set torch to use deterministic algorithm



In [4]:
seed = 0

torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Set the environment variable for deterministic CuBLAS operations
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

# Enable deterministic algorithms in PyTorch
torch.use_deterministic_algorithms(False)
# torch.backends.cudnn.benchmark = False # maybe cancell warning: UserWarning: Plan failed with a cudnnException, not tested

device = "cuda:0" if torch.cuda.is_available() else "cpu"

### Color Class
This class is used for printing text in different color. In particular each attribute in the Color class corresponds to a specific text color or formatting style.

In [5]:
class Color:
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    MAGENTA = '\033[95m'
    CYAN = '\033[96m'
    WHITE = '\033[97m'
    RESET = '\033[0m' #RESET: Resets all text formatting and color to default
    BOLD = '\033[1m' #BOLD: Makes the text bold
    UNDERLINE = '\033[4m' #UNDERLINE: Underlines the text

Random Seed

# Dataset Setup

## Initial setup


The function, given the link of the dataset in google drive, download it.

Args:
        
1.   **link_dataset** = link to zip files
2.   **delete_zip_file** = {True/False}, use to remove the zip file once extracted the content

Note: the link my be public or gdown can't download it


In [6]:
def download_dataset(link_dataset, delete_zip_file=False):

    # Creating string for download

    link_download_base =  global_var['link_download_prefix'] # General link for the download from Drive (with "YOUR_FILE_ID")
    id_dataset = link_dataset.split("/view")[0].split("/")[-1]
    dataset_download_link = link_download_base.replace("YOUR_FILE_ID", id_dataset) # Replace the "YOUR_FILE_ID" part with the extracted id_dataset

    path_download_dataset = global_var['path_download_dataset'] # Path of the zip file that contain the dataset
    dataset_name = global_var['dataset_name'] # Dataset's Name
    extract_directory = global_var['extract_directory'] # Directory of the GColab Runtime where we want to extract the dataset

    # Downloading zip file
    try:
        if not( os.path.exists(dataset_name) ): # check is the dataset is already downloaded
            gdown.download(
                url=dataset_download_link, # Link that we extract with the file ID
                output=path_download_dataset, # Path of the runtime Colab to save the zip file
                quiet=False # Output to check the download
            )
            print(f"{Color.GREEN}\nZip file downloaded{Color.RESET}")
        else:
            print(f"{Color.GREEN}Zip file already downloaded{Color.RESET} ")
    except Exception as error:
        print(f"{Color.RED}Download went wrong!\nerror:{Color.RESET}")
        print(error)


    # Unzipping the file
    '''
    In this part we want to unzip the zip file contain the dataset
    '''
    try:
        if not( os.path.exists(dataset_name) ):

          # This open the file zip in path_download_dataset in 'r' = read mode.
          # The file is created from the ZipFile class contained in zipfile module
          # and the name of the instance will be zipfile

            with zipfile.ZipFile(path_download_dataset, 'r') as zip_file:

                n_files = len(zip_file.namelist()) # number of files and directories inside the zip

                with tqdm(total=n_files, desc='Unzipping files') as pbar: # It's a bar to track the unzip process, we pass the number of total files
                    for file_name in zip_file.namelist():
                        zip_file.extract(member=file_name, path=extract_directory) # Extract the file_name (iteration over all the files) in the path extract_directory
                        pbar.update(1) # increment the progress bar of 1 unit for each extraction

            print(f"{Color.GREEN}Dataset {dataset_name} unzipped{Color.RESET}")
        else:
            print(f"{Color.GREEN}Dataset {dataset_name} already unzipped{Color.RESET} ")
    except Exception as error:
        print(f"{Color.RED}Unzip went wrong!\nerror:{Color.RESET}")
        print(error)

    # Removing zip file
    '''
    In this part we remove the zip file if the flag
    delete_zip_file is True (args of the function)
    '''
    try:
        if delete_zip_file == True:
            ! rm {path_download_dataset}
            print(f"{Color.GREEN}File {path_download_dataset} removed{Color.RESET}")
    except Exception as error:
        print(f"{Color.RED}Could not remove zip file, pass!\nerror:{Color.RESET}")
        print(error)
        pass

**Merge Dataset**

Given the dataset, it merge and shuffle it.

In [7]:
def merge_dataset():

    dataset_name = global_var['dataset_name'] # Name of the directory
    extract_directory = global_var['extract_directory'] # Directory of destination where we put the shuffled dataset
    current_path = os.path.join(extract_directory, dataset_name) # Create the path "/content/chest_xray"
    origin_dataset = os.path.join(current_path,dataset_name) # Create the path "/content/chest_xray/chest_xray"

    # Create a list of this type -> ['/content/chest_xray/train', '/content/chest_xray/test', '/content/chest_xray/val']:
    sub_directory = [ os.path.join(current_path, s) for s in ['train', 'test', 'val']]
    classes = ['PNEUMONIA', 'NORMAL']

    try:
        for sub_dir in sub_directory:

            for c in classes:
                sub_dir_c = os.path.join(sub_dir, c) # Create a path of the type '/content/chest_xray/train/PNEUMONIA' and '/content/chest_xray/train/NORMAL' and so on...
                counter = 0 # This is used for the name of each image

                for img_name in os.listdir(sub_dir_c):
                    counter += 1
                    # Example of img_name:  IM-0001-0001.jpeg

                    # sub_dir.split("/")[-1]: Take the part "train", "test" or "val"
                    # c.lower(): Take the name of the class in uppercase and transform in lowercase
                    # f"_{counter:05d}": put the number counter starting with a zero sequence until 5 numbers sequence
                    # current_name.split(".")[-1]: put the jpeg word at the end
                    # Example of new_name: train_pneumonia_00001.jpeg

                    new_name = sub_dir.split("/")[-1] + "_" + c.lower() + f"_{counter:05d}" + "." + img_name.split(".")[-1]

                    final_path = sub_dir + "/" + new_name # Create a path of the type: /content/chest_xray/train/train_pneumonia_00001.jpeg

                    # Move the file from '/content/chest_xray/train/PNEUMONIA/IM-0001-0001.jpeg' to '/content/chest_xray/train/train_pneumonia_00001.jpeg'
                    # this result in changin the name of the file
                    os.rename( os.path.join(sub_dir_c, img_name), final_path )

                ! rmdir {sub_dir_c} # Remove the empty directory "PNEUMONIA" and "NORMAL" after all rename
        !rm -rf {origin_dataset} # Remove a useless directory

        print(f"{Color.GREEN}Dataset reordered and labelled{Color.RESET}")
    except Exception as error:
        print(f"Dataset already merged\n")
        pass

## Counter Function
This function is used in two options:

1. **labels** = *True* : in this case the counter will count the number of "normal" or "pneumonia" images in the "path_dir"

1. **labels** = *False* : in this case the counter will count simply the number of images in the "path_dir"

In [8]:
def counter (path_dir, labels = True):

  file_list = os.listdir(path_dir) # Create a list of all files in the dataset directory

  if labels == True:

    train_count_normal = 0 # Counter of images "normal"
    train_count_pneumonia = 0 # Counter of images "pneumonia"

  # Iterate through each file in the directory
    for filename in file_list:
        if 'normal' in filename:
            train_count_normal += 1
        elif 'pneumonia' in filename:
            train_count_pneumonia += 1

    return train_count_normal, train_count_pneumonia

  else:

    samples_num = len(file_list);
    return samples_num

# Preprocessing operation on dataset

In [9]:
def get_gaussian(img_original):

    img_original = cv2.imread(img_original, cv2.IMREAD_GRAYSCALE)

    smoothing_value = global_var['smoothing_value']
    img_smoothed = cv2.GaussianBlur( img_original, ( smoothing_value, smoothing_value), 0 )

    return img_smoothed

In [10]:
def get_gaussian_clahe(img_original):

    img_original = cv2.imread(img_original, cv2.IMREAD_GRAYSCALE)

    smoothing_value = global_var['smoothing_value']
    img_smoothed = cv2.GaussianBlur( img_original, ( smoothing_value, smoothing_value), 0 )

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_eq = clahe.apply(img_smoothed)

    return img_eq

In [11]:
def get_gaussian_he(img_original):

    img_original = cv2.imread(img_original, cv2.IMREAD_GRAYSCALE)

    smoothing_value = global_var['smoothing_value']
    img_smoothed = cv2.GaussianBlur( img_original, ( smoothing_value, smoothing_value), 0 )
    img_eq = cv2.equalizeHist(img_smoothed )

    return img_eq

In [12]:
def resize_and_save(mode, size):

    original_dataset = global_var['dataset_name'] # Name of the original dataset
    resized_dataset = global_var['dataset_name_resized'] # Name of the resized dataset
    sub_folders_ttv = global_var['sub_folders_ttv'] # sub folder 'train', 'test', 'val'

    extract_directory = global_var['extract_directory'] # Directory of the GColab Runtime where we want to extract the dataset
    original_path = os.path.join(extract_directory, original_dataset) # Create the path "/content/chest_xray"
    resized_path = os.path.join(extract_directory, resized_dataset) # Create the path "/content/chest_xray_resized"

    # Create a list of this type -> ['/content/chest_xray/train', '/content/chest_xray/test', '/content/chest_xray/val']:
    sub_directory_original = [ os.path.join(original_path, s) for s in sub_folders_ttv]

    try:
      for sub_dir in sub_directory_original:
        category = os.path.basename(sub_dir) # Extract the part 'train', 'test' or 'val'

        # Copy from source to destination
        source_dir = sub_dir # Source Directory (Ex. '/content/chest_xray/train')
        destination_dir = resized_path + '/' + category # Destination Directory ('Ex. /content/chest_xray_resized/train')

        if not(os.path.exists(destination_dir)):
          # Copy the entire directory
          shutil.copytree(source_dir, destination_dir)

          # Loop for the resizing and save
          img_files = os.listdir(sub_dir)
          for img in tqdm(img_files, desc=f"Resizing {category} images", unit="image"):

            # Take the name and extension of the image
            name, extension = os.path.splitext(img)

            # Read the image
            img_path = sub_dir + '/' + name + extension
            img = read_image(img_path) # Tensor Image (Ex. torch.Size([1, 928, 1288]))

            # Add a batch dimensione of 1 needed for the nn.functional.interpolate method
            img_4d = img.unsqueeze(0) # Tensor Image (Ex. torch.Size([1, 1, 928, 1288]))

            # Interpolation method
            resized_image = nn.functional.interpolate(img_4d, size=size, mode=mode) # Tensor Image (Ex. torch.Size([1, 1, 800, 800]))

            # Remove the batch dimension
            resized_image = resized_image.squeeze(0)  # Tensor Image (Ex. torch.Size([1, 800, 800]))

            # Permute the dimensions
            resized_image = resized_image.permute(1, 2, 0) # Tensor Image (Ex. torch.Size([800, 800, 1]))

            # Transform in numpy structure
            resized_image = resized_image.numpy() # Numpy Image (Ex. (800, 800, 1))

            # Convert to PIL image using torchvision.transforms
            to_pil = ToPILImage() # Transformer Numpy -> PIL Image
            resized_image_pil = to_pil(resized_image) # PIL Image

            # Save the resulting image
            path_to_save = destination_dir + '/' + name + extension # Path of the type '/content/chest_xray_resized/train/img_name.jpeg'
            resized_image_pil.save(path_to_save) # Function to save the image
        else:
          print(f"{Color.GREEN}Data for {category} already resized{Color.RESET} ")

    except Exception as error:
      print(f"Problem in resizing\n")
      print(error)
      pass

In [13]:
def data_augmentation(transform):

    tr = transform # Transform that we use for augment the data of the training set
    original_dataset = global_var['dataset_name_resized'] # Name of the original dataset
    new_dataset = global_var['dataset_name_resized_augmented'] # Name of the resized dataset
    sub_folders_vtt = global_var['sub_folders_ttv'] # sub folder 'train', 'test', 'val'

    extract_directory = global_var['extract_directory'] # Directory of the GColab Runtime where we want to extract the dataset
    original_path = os.path.join(extract_directory, original_dataset) # Create the path "/content/chest_xray_resized"
    new_path = os.path.join(extract_directory, new_dataset) # Create the path "/content/chest_xray_resized_augmented"

    # Create a list of this type -> ['/content/chest_xray_resized/train', '/content/chest_xray_resized/test', '/content/chest_xray_resized/val']:
    sub_directory_original = [ os.path.join(original_path, s) for s in sub_folders_vtt]
    # Create a list of this type -> ['/content/chest_xray_resized_augmented/train', '/content/chest_xray_resized_augmented/test', '/content/chest_xray_resized_augmented/val']:
    sub_directory_augmented = [ os.path.join(new_path, s) for s in sub_folders_vtt]

    try:
      for sub_dir in sub_directory_original:
        category = os.path.basename(sub_dir) # Extract the part 'train', 'test' or 'val'

        # Copy from source to destination
        source_dir = sub_dir # Source Directory ('Ex. /content/chest_xray_resized/train')
        destination_dir = new_path + '/' + category # Destination Directory ('Ex. /content/chest_xray_resized_augmented/train')

        if not(os.path.exists(destination_dir)):
          # Copy the entire directory
          shutil.copytree(source_dir, destination_dir)
        else:
          print(f"{Color.GREEN}Data for {category} already copied{Color.RESET} ")
          return

    except Exception as error:
      print(f"Problem in copying\n")
      print(error)
      pass

    try:
      dir_train = sub_directory_augmented[0]                   # '/content/chest_xray_resized_augmented/train'
      img_files_train = os.listdir(dir_train)                  # List of all training images
      n_normal, n_pneumonia = counter(dir_train, labels=True)  # Count the number of normal and pneumonia images
      difference = n_pneumonia - n_normal                      # Desired value of augmented images
      sub_path = os.path.join(dir_train, '/augmented')         # Subdirectory where we save the augmented images
      !mkdir {dir_train + '/augmented/'}                       # Create the subdirectory
      actual = len(os.listdir(dir_train + '/augmented/'))      # Actual value of augmented images

      n = 0                                                    # Counter used to name the augmented images

      with tqdm(total=difference, desc='Balancing data') as pbar:
        while  actual < difference:
          img = random.choice(img_files_train)           # Pick a random image file from the list
          if ('normal' in img):
            n += 1
            # Take the name and extension of the image
            name, extension = os.path.splitext(img)

            # Read the image as a Tensor
            img = read_image(dir_train + '/' + img)

            # Transform the image
            new_image = transform(img); # Transformed Tensor Image

            # Permute the dimensions
            new_image = new_image.permute(1, 2, 0) # Tensor Image (Ex. torch.Size([800, 800, 1]))

            # Transform in numpy structure
            new_image = new_image.numpy() # Numpy Image (Ex. (800, 800, 1))

            # Convert to PIL image using torchvision.transforms
            to_pil = ToPILImage() # Transformer Numpy -> PIL Image
            new_image_pil = to_pil(new_image) # PIL Image

            # Save the resulting image:  Path of the type /content/chest_xray_resized_augmented/train/augmented/train_normal_00001_augmented.jpeg
            path_to_save = dir_train + '/augmented/' + name.split('_')[0] + '_' + name.split('_')[1] + f"_{actual:05d}" + '_augmented' + extension
            new_image_pil.save(path_to_save) # Function to save the image
            actual += 1     # Update of Actual value of agumented images
            pbar.update(1)

      for img_name in os.listdir(dir_train + '/augmented/'):
        final_path = dir_train + '/' # Create a path of the type: /content/chest_xray_resized_augmented/train/
        os.rename(dir_train + '/augmented/' + img_name, final_path + img_name)

      !rmdir {dir_train + '/augmented/'} # Remove the empty directory "PNEUMONIA" and "NORMAL" after all rename

    except Exception as error:
        print(f"Problem in augmenting\n")
        print(error)
        pass

In [14]:
def augment_class(class_to_augment, n_new_samples, transform, subfolder_to_augment):

    if n_new_samples == 0:
      return

    resized_dataset = global_var['dataset_name_resized'] # where take imaged to augment
    augmented_dataset = global_var['dataset_name_resized_augmented'] # where add new samples
    dir_train = os.path.join(resized_dataset, subfolder_to_augment)
    image_list = os.listdir( dir_train )

    actual_id = len(image_list)

    with tqdm(total=n_new_samples, desc=f'Augmenting data for {class_to_augment} ') as pbar:

      while actual_id < n_new_samples + len(image_list) :
        img = random.choice( image_list )

        if class_to_augment in img:
          # Take the name and extension of the image
          name, extension = os.path.splitext(img)

          # Read the image as a Tensor
          img = read_image(dir_train + '/' + img)

          # Transform the image
          new_image = transform(img); # Transformed Tensor Image

          # Permute the dimensions
          new_image = new_image.permute(1, 2, 0) # Tensor Image (Ex. torch.Size([800, 800, 1]))

          # Transform in numpy structure
          new_image = new_image.numpy() # Numpy Image (Ex. (800, 800, 1))

          # Convert to PIL image using torchvision.transforms
          to_pil = ToPILImage() # Transformer Numpy -> PIL Image
          new_image_pil = to_pil(new_image) # PIL Image

          # Save the resulting image
          path_to_save = augmented_dataset + f'/{subfolder_to_augment}/' + name.split('_')[0] + '_' + name.split('_')[1] + f"_{actual_id:05d}" + '_augmentedv2' + extension
          new_image_pil.save(path_to_save)

          pbar.update(1)
          actual_id += 1

In [15]:
def create_subset(main_path, subfolder_percentage):

  substring = global_var["subset_suffix"]
  classes = ["normal", "pneumonia"]
  sub_dir = global_var['sub_folders_ttv'] # train, test, val
  subs_paths = [ os.path.join(main_path + "_subset", subs) for subs in sub_dir ]

  if os.path.exists(main_path + "_subset"):
    print(f"{Color.GREEN}Subset already created{Color.RESET}")
    return

  for s in subs_paths:
    if not os.path.exists(s):
      os.makedirs(s)

  subfolder_dimension = {}
  for s_path in os.listdir(main_path):
      num_files = len( os.listdir( os.path.join(main_path, s_path)) )  # compute actual dimension of subfolder

      subfolder_size = round(num_files * subfolder_percentage)  # compute new dimension
      subfolder_dimension[s_path] = subfolder_size


  for subs_path in subs_paths:  # iterate over train, test, val on subset
    subs_counter = 0
    sub_dir_type = subs_path.split("/")[-1] # take the type train, test, val
    image_list = os.listdir( os.path.join(main_path, sub_dir_type) ) # list of all images

    for c in classes: # iter over classes to ensure a balanced subset
      while len(os.listdir(subs_path)) < subfolder_dimension[sub_dir_type]*(subs_counter +1):  # check on the length on the subdir
        img = random.choice( image_list )


        if c in img: # copying images
          source_img_path = os.path.join(main_path, sub_dir_type, img)
          dest_img_path = os.path.join(subs_path, img)
          shutil.copy(source_img_path, dest_img_path)

      subs_counter += 1

    print(f"{Color.GREEN}Subset {subs_path} created, {Color.MAGENTA}{subfolder_dimension[sub_dir_type]} samples {Color.GREEN}per class{Color.RESET}")


  total_samples = 0
  for subs_path in subs_paths:
    total_samples += len(os.listdir(subs_path))

  print(f"{Color.GREEN}Subset created with a total of {Color.MAGENTA}{total_samples} samples{Color.RESET}")

In [16]:
def unify_folder(path_to_folder):

  sub_folders = global_var['sub_folders_ttv'] # ['train', 'test', 'val']

  # check if already unified
  if len(os.listdir(path_to_folder)) != len(sub_folders):
    print(f"{Color.GREEN}Already unified{Color.RESET}")
    return

  for sub_folder in sub_folders:

    if not os.path.exists(os.path.join(path_to_folder, sub_folder)):
      print(f"{Color.RED}Subfolder {sub_folder} doesn't exist{Color.RESET}")
      continue

    for images in os.listdir( os.path.join( path_to_folder, sub_folder) ):

      path_to_img = os.path.join(path_to_folder, sub_folder, images)
      shutil.move(path_to_img, path_to_folder)

    shutil.rmtree(os.path.join(path_to_folder, sub_folder))

  print(f"{Color.GREEN}Folder {path_to_folder} unified{Color.RESET}")

In [17]:
def preprocess_image(image_global_path, mode):

    # based of the selected mode, it does a preprocess on image
    # returun None is something went wrong

    if mode == 'gaussian_clahe':
        preprocessed_img = get_gaussian_clahe( image_global_path )
    elif mode == 'gaussian':
        preprocessed_img = get_gaussian( image_global_path )
    elif mode == 'gaussian_he':
        preprocessed_img = get_gaussian_he( image_global_path )
    else:
        print(f"{Color.RED}Preprocessing {mode} is not available{Color.RESET}")
        ! rm -rf {new_dataset_path}

        preprocessed_img = None

    return preprocessed_img

In [18]:
def create_preprocessed_dataset( dataset_path, new_dataset_path, mode ):

    categories = global_var['sub_folders_ttv'] #['train', 'test', 'val']
    mode = mode.lower()

    if os.path.exists(new_dataset_path):
        print(f"{Color.GREEN}Dataset {new_dataset_path} images already exists{Color.RESET}")
        return
    else:
        ! cp -r {dataset_path} {new_dataset_path}   # coping augmented dataset "/content/chest_xray_{mode}"

    for c in categories:  # iterate on train, test, val

        n_files = len( os.listdir( os.path.join(new_dataset_path, c ) ) )   # it is "/content/chest_xray_{mode}/train,test,val"

        with tqdm(total=n_files, desc='Processing ' + c) as pbar:
            for img_path in os.listdir( os.path.join(dataset_path, c) ):   # iterate on train, test, val of heatmap dataset

                full_img_path = new_dataset_path + "/" + c + "/" + img_path # global path to image

                preprocessed_img = preprocess_image(full_img_path, mode)    # preprocess the image according the the selected mode

                '''
                if "augmented_v2" in img_path:
                    new_name = img_path.replace("augmented_v2", mode)   # renaming the images
                elif "augmented" in img_path:
                    new_name = img_path.replace("augmented", mode)   # renaming the images
                else:
                    new_name = img_path.replace(".", f"_{mode}.")
                '''
                new_name = img_path.replace(".", f"_{mode}.")

                #print(new_name)
                new_full_path = full_img_path.replace(img_path, new_name)  # /content/chest_xray_heatmap/train/test_normal_00001_heatmap.jpeg

                cv2.imwrite(full_img_path, preprocessed_img) # overwrite the image
                os.rename(full_img_path, new_full_path) # renaming the image

                pbar.update(1)

# Dataset and Network classes

## ChestRayDataset

In [19]:
class ChestRayDataset(Dataset):

    def __init__(self,file_path, transform):
        self.file_path = file_path
        self.transform = transform

    def __len__(self):
        return len(os.listdir(self.file_path))

    def __getitem__(self, index):
        sample_path = self.file_path + "/" + os.listdir(self.file_path)[index]
        sample = cv2.imread(sample_path)
        sample = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)

        # Output of the class
        sample_tensor = self.transform(sample)
        label = sample_path.split("/")[-1].split("_")[1]

        if label == "normal":
            label = 0
        else:
            label = 1

        return sample_tensor,label

## Architecture

### Blocks

In [20]:
class SeparableConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=7, padding='same', bias=True):
        super(SeparableConv2D, self).__init__()

        self.depthwise = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            padding=padding,
            groups=in_channels,
            bias=bias
            )

        self.pointwise = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=1,
            bias=bias
            )

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

In [21]:
class CAM(nn.Module):
    def __init__(self, channels, ratio=8, bias=False):
        super(CAM,self).__init__()

        self.fc1 = nn.Linear(
            in_features=channels,
            out_features=channels//ratio,
            bias=bias
            )

        self.fc2 = nn.Linear(
            in_features=channels//ratio,
            out_features=channels,
            bias=bias
            )

        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.global_max_pool = nn.AdaptiveMaxPool2d((1,1))

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        # Input Tensor: torch.Size([1, 1024, 8, 8])
        x1 = torch.mean(x, dim=(2,3)) # GAP: torch.Size([1, 1024])
        x1 = self.fc1(x1) # GAP + Dense: torch.Size([1, 128])
        x1 = self.relu(x1)
        x1 = self.fc2(x1) # GAP + Dense + Dense: torch.Size([1, 1024])


        x2 = self.global_max_pool(x) # GMP: torch.Size([1, 1024, 1, 1])
        x2 = x2.view(x2.size(0), -1) # view: torch.size([1, 1024])
        x2 = self.fc1(x2) # GMP + Dense: torch.Size([1, 128])
        x2 = self.relu(x2)
        x2 = self.fc2(x2) # GMP + Dense + Dense: torch.Size([1, 1024])

        feats = x1 + x2 # Feats shape: torch.Size([1, 1024])
        feats = self.sigmoid(feats)
        feats = feats.view(feats.size(0), feats.size(1), 1, 1) # feats.shape: torch.Size([1, 1024, 1, 1])
        feats = torch.mul(x, feats) # Output shape: torch.Size([1, 1024, 8, 8])
        return feats

In [22]:
class SAM_avg (nn.Module):
    def __init__(self, channels, padding='same'):

        super(SAM_avg, self).__init__()

        self.sep_conv2d_1 = SeparableConv2D(
            in_channels=channels,
            out_channels=channels,
            kernel_size=1,
            padding=padding
        )

        self.sep_conv2d_2 = SeparableConv2D(
            in_channels=channels,
            out_channels=channels,
            kernel_size=3,
            padding=padding
        )

        self.batchnorm2d = nn.BatchNorm2d(channels)

        self.conv2d_3 = nn.Conv2d(
            in_channels=1,
            out_channels=1,
            kernel_size=7,
            padding=padding
        )

        self.sigm= nn.Sigmoid()

    def forward(self, x, cam):


        # Input tensor: torch.Size([1, 1024, 8, 8])

        x = self.sep_conv2d_1(x) # SepConv2d_1 shape: torch.Size([1, 1024, 8, 8])
        x = self.sep_conv2d_2(x) # SepConv2d_2 shape: torch.Size([1, 1024, 8, 8])
        x = self.batchnorm2d(x) # BatchNorm2d shape: torch.Size([1, 1024, 8, 8])
        x = x*cam # X*CAM shape: torch.Size([1, 1024, 8, 8])

        x1 = torch.mean(x, dim=1, keepdim=True) # Mean pooling shape: torch.Size([1, 1, 8, 8])

        feats = self.conv2d_3(x1) # Conv2d shape: torch.Size([1, 1, 8, 8])
        feats = self.sigm(feats)
        out = x * feats # Output shape: torch.Size([1, 1024, 8, 8])

        return out

In [23]:
class SAM_max (nn.Module):
    def __init__(self, channels, padding='same'):

        super(SAM_max, self).__init__()  # Initialize nn.Module

        self.sep_conv2d_1 = SeparableConv2D(
            in_channels=channels,
            out_channels=channels,
            kernel_size=1,
            padding=padding
        )

        self.sep_conv2d_2 = SeparableConv2D(
            in_channels=channels,
            out_channels=channels,
            kernel_size=3,
            padding=padding
        )

        self.batchnorm2d = nn.BatchNorm2d(channels)

        self.conv2d_3 = nn.Conv2d(
            in_channels=1,
            out_channels=1,
            kernel_size=7,
            padding=padding
        )

        self.sigm= nn.Sigmoid()

    def forward(self, x, cam):

        # input: torch.Size([1, 1024, 8, 8])

        x = self.sep_conv2d_1(x) # SepConv2d_1 shape: torch.Size([1, 1024, 8, 8])
        x = self.sep_conv2d_2(x) # SepConv2d_2 shape: torch.Size([1, 1024, 8, 8])
        x = self.batchnorm2d(x) # BatchNorm2d shape: torch.Size([1, 1024, 8, 8])
        x = x*cam # X*CAM shape: torch.Size([1, 1024, 8, 8])

        # Max Pooling
        x2, _ = torch.max(x, dim=1, keepdim=True) # MAX pooling shape: torch.Size([1, 1, 8, 8])

        # Convolution and Multiplication
        feats = self.conv2d_3(x2) # Conv2d shape: torch.Size([1, 1, 8, 8])
        feats = self.sigm(feats)
        out = x*feats # Output shape: torch.Size([1, 1024, 8, 8])

        return out

In [24]:
class RichardsSigmoid(nn.Module):
    def __init__(self, units=1, bias=True):
        super(RichardsSigmoid, self).__init__()

        self.A = nn.Parameter(torch.Tensor(units))
        self.Q = nn.Parameter(torch.Tensor(units))
        self.mu = nn.Parameter(torch.Tensor(units))

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.zeros_(self.A)
        nn.init.zeros_(self.Q)
        nn.init.zeros_(self.mu)

    def forward(self, x):
        # Richards sigmoid function
        return 1 / (1 + torch.exp(-self.A * torch.exp(-self.Q * (x - self.mu))))

    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (self.units,)

In [25]:
class ChannelDropout(nn.Module):
    def __init__(self, channels, drop_ratio=0.2):
        super(ChannelDropout, self).__init__()

        self.drop_ratio = drop_ratio # Ratio of channels to be dropped.

        # Build the Mask with ones
        self.mask = nn.Parameter(torch.ones(1, channels, 1 , 1))

        # Build the Richard Sigmoid to apply to the mask
        self.r_sigm = RichardsSigmoid(units=1)

    def forward(self, x):
        # Apply the Richard Sigmoid to the mask
        mask = self.r_sigm(self.mask) # MASK Shape: torch.Size([1, 3072, 1, 1])

        # Repeat the mask as the batch size of the input (to apply to all the features)
        batch_size = x.size(0)
        mask_replicated = mask.repeat(batch_size, 1, 1, 1)

        # Apply the mask to the input
        x = x * mask_replicated # Input after Mask: torch.Size([1, 3072, 8, 8])

        # New Channel dimension
        new_n_channels = int(x.shape[1] * (1 - self.drop_ratio))

        # Perform top-k operation along the channels dimension (dim=1)
        # Keep the channels with highest values
        _, indices = torch.topk(x, k=new_n_channels, dim=1, largest=True)
        output = torch.gather(x, dim=1, index=indices)

        return output

In [26]:
class CSSAM (nn.Module):
    def __init__(self, channels, padding='same', ratio=8, bias=False, drop_ratio=0.2):

        super(CSSAM, self).__init__()  # Initialize nn.Module


        self.SamAvg = SAM_avg(
            channels=channels,
            padding=padding
            )

        self.SamMax = SAM_max(
            channels=channels,
            padding=padding
            )

        self.cam = CAM(
            channels=channels,
            ratio=ratio,
            bias=bias
            )

        self.ch_dropout = ChannelDropout(
            channels=channels*3,
            drop_ratio=drop_ratio
            )

    def forward(self, x):

        cam = self.cam(x)

        # SAM average and SAM max
        x_avg = self.SamAvg(x, cam) # SAM_avg shape: torch.Size([1, 1024, 8, 8])
        x_max = self.SamMax(x, cam) # SAM_max shape: torch.Size([1, 1024, 8, 8])

        # Concatenation
        x = torch.cat((x_avg, x_max, cam), dim=1) # Concatenation shape: torch.Size([1, 3072, 8, 8])

        # Channel Dropout
        out = self.ch_dropout(x) # Channel Dropout shape: torch.Size([1, 1536, 8, 8])

        return out

### Bodies

In [27]:
class JAM_network(nn.Module):
    def __init__(self, channels, padding='same', drop_ratio=0.2, dropout_rate=0.35 ):
        super(JAM_network,self).__init__()

        self.densenet169 = models.densenet169(pretrained=True).features

        self.cssam = CSSAM(
            channels=channels,
            padding=padding,
            drop_ratio=drop_ratio
            )

        self.dropout = nn.Dropout(
            p=dropout_rate
            )


        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)


    def forward(self, x):

        # Input Shape: torch.Size([1, 3, 256, 256])

        # BACKBONE
        x_dense = self.densenet169(x) # x_dense.shape: torch.Size([1, 1024, 8, 8])

        # CSSAM
        x_cssam = self.cssam(x_dense) # CSSAM shape: torch.Size([1, 2457, 8, 8])

        # GAP
        feats = self.global_avg_pool(x_cssam) # GAP shape: torch.Size([1, 2457, 1, 1])
        feats = torch.flatten(feats, start_dim=1) # Flattening shape: torch.Size([1, 2457])

        # DROPOUT
        feats = self.dropout(feats) # dropout: torch.Size([1, 2457])

        return feats

In [28]:
class JAM(nn.Module):
    def __init__(self, channels, device, model_name, padding='same', drop_ratio=0.2, dropout_rate=0.35):
        super().__init__()

        self.body = JAM_network(
            channels=channels,
            padding=padding,
            drop_ratio=drop_ratio,
            dropout_rate=dropout_rate
            ).to(device)

        self.kan_ch = int(channels * 3 * (1 - drop_ratio))
        self.classifier = KAN([ self.kan_ch, 1])
        self.classifier.to(device)

        self.sig = nn.Sigmoid()

        self.loss_fn = nn.BCELoss()
        self.optimizer = torch.optim.AdamW(self.parameters(), lr=0.0001) # torch.optim.SGD(self.model.parameters(), lr=0.1)

        self.accuracy = BinaryAccuracy()
        self.precision = BinaryPrecision()
        self.recall = BinaryRecall()
        self.auroc = BinaryAUROC()
        self.f1score = BinaryF1Score()

        self.model_name = model_name

    def set_training_mode(self):
        self.body.train()

    def set_eval_mode(self):
        self.body.eval()

    def forward(self, model_input):
        out_body = self.body(model_input)
        out_cls = self.classifier(out_body)
        out = self.sig(out_cls)
        return out

    def save(self, string_id, stamp=True):
        name = self.model_name + f"_{string_id}.pt"
        torch.save(self.state_dict(), name )

        if stamp==True:
            print(f"{Color.MAGENTA}saved: {name}{Color.RESET}")

    def load(self, string_id):
        name = self.model_name + f"_{string_id}.pt"
        try:
            self.load_state_dict(torch.load(name) )
            print(f"{Color.MAGENTA}loaded: {name}{Color.RESET}")
        except Exception as e:
            print(f"{Color.RED}Model not loaded: try specify 'string_id' parameter{Color.RESET}")
            print(e)

'''
print(device)
jam_try = JAM(1664, device=device, model_name="jam_model.pt")
t = torch.rand([2, 3, 256, 256]).to(device)
print(f"input tensor: {t.shape}")

out = jam_try(t)
print(f"out.shape: {out.shape},\n{out}")
'''

'\nprint(device)\njam_try = JAM(1664, device=device, model_name="jam_model.pt")\nt = torch.rand([2, 3, 256, 256]).to(device)\nprint(f"input tensor: {t.shape}")\n\nout = jam_try(t)\nprint(f"out.shape: {out.shape},\n{out}")\n'

# Dataset informations

## Download and merge of the dataset

In [29]:
download_dataset(
     link_dataset=global_var['link_dataset'],
     delete_zip_file=False
 )

merge_dataset()

Downloading...
From (original): https://drive.google.com/uc?export=download&id=10un_W7teXQy9LOB1uqd0e6VJ46UFNRza
From (redirected): https://drive.google.com/uc?export=download&id=10un_W7teXQy9LOB1uqd0e6VJ46UFNRza&confirm=t&uuid=229a8717-d25f-41d4-9cc9-f82669f85b64
To: /content/dataset.zip
100%|██████████| 2.46G/2.46G [01:10<00:00, 35.0MB/s]


Zip file downloaded


Unzipping files:   0%|          | 0/17591 [00:00<?, ?it/s]

Dataset chest_xray unzipped
Dataset reordered and labelled


In [ ]:
size = global_var["resizing_dim"] # Choose the size

resize_and_save(
    mode='bilinear',    # Choose the mode of interpolation ('nearest', 'bilinear', 'bicubic', ...)
    size=size
)

Resizing train images:   0%|          | 0/5216 [00:00<?, ?image/s]

In [ ]:
probs = 0.15

resize_crop_transform = transforms.RandomResizedCrop(size=size, scale=(0.8, 1.0))  # For zoom and size change
rotation_transform = transforms.RandomRotation(degrees=(0,20))  # For random rotation
horizontal_flip_transform = transforms.RandomHorizontalFlip(p=1.0)  # For horizontal flip
shift_transform = transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))  # for random shift beween 0 and translate value


tr_augmentation = transforms.Compose([
     transforms.RandomApply( [resize_crop_transform], p=probs ),
     transforms.RandomApply( [rotation_transform], p=probs ),
     transforms.RandomApply( [horizontal_flip_transform], p=probs ),
     transforms.RandomApply([shift_transform], p=probs),
])

data_augmentation(tr_augmentation)

if global_var["augment_performed"] == False:
    aug_train_dimension = global_var["augmentation_train_dimension"]
    aug_test_dimension = global_var["augmentation_test_dimension"]

    augment_class('normal', aug_train_dimension, tr_augmentation, 'train')
    augment_class('pneumonia', aug_train_dimension, tr_augmentation, 'train')

    augment_class('normal', aug_test_dimension, tr_augmentation, 'test')

    global_var["augment_performed"] = True

In [ ]:
dataset_directory = global_var["extract_directory"] + "/" + global_var["dataset_name_resized_augmented"]
sub_directory = global_var['sub_folders_ttv']


for sub_dir in sub_directory:
  pneumonia_counter = 0
  normal_counter = 0
  for img in os.listdir( os.path.join (dataset_directory, sub_dir) ):
    if "pneumonia" in img:
      pneumonia_counter += 1
    else:
      normal_counter += 1

  print(f"The number of pneumonia images in {sub_dir}:", pneumonia_counter)
  print(f"The number of normal images in {sub_dir}:", normal_counter)

In [ ]:
dataset_directory = global_var["extract_directory"] + "/" + global_var["dataset_name_resized_augmented"]
gaussian_he_directory = global_var["extract_directory"] + "/" + global_var["dataset_name_gaussian_he"]

create_preprocessed_dataset(
    dataset_path = dataset_directory,
    new_dataset_path = gaussian_he_directory,
    mode = 'gaussian_he'
)

In [ ]:
dataset_directory = global_var["extract_directory"] + "/" + global_var["dataset_name_resized_augmented"]
gaussian_directory = global_var["extract_directory"] + "/" + global_var["dataset_name_gaussian"]

create_preprocessed_dataset(
    dataset_path = dataset_directory,
    new_dataset_path = gaussian_directory,
    mode = 'gaussian'
)

In [ ]:
dataset_directory = global_var["extract_directory"] + "/" + global_var["dataset_name_resized_augmented"]
gaussian_clahe_directory = global_var["extract_directory"] + "/" + global_var["dataset_name_gaussian_clahe"]

create_preprocessed_dataset(
    dataset_path = dataset_directory,
    new_dataset_path = gaussian_clahe_directory,
    mode = 'gaussian_clahe'
)

## Dataset exploration, augmentation and visualization

In this part we plot an:

1. *Histogram Distribution* : to see if the dataset is balanced or unbalanced

2. *Pie Chart* : to visualize the sizes of training, test and validation sets

In [ ]:
# @title Initial statistic of dataset
# Definition of the paths for training, test and validation set
path_training_set = "/content/chest_xray/train"
path_test_set = "/content/chest_xray/test"
path_val_set = "/content/chest_xray/val"

# Counting the Normal and Pneumonia images in the training set
train_norm, train_pneu = counter(path_training_set, labels = True); # Count the number of normal images and pneumonia images

train_num = counter(path_training_set, labels = False);
test_num = counter(path_test_set, labels = False);
val_num = counter(path_val_set, labels = False);

# Histogram Data
classes_hist = ['Normal', 'Pneumonia']
counts_hist = [train_norm, train_pneu]
colors_hist = ['aquamarine', 'blue']

# Pie Chart Data
classes_pie = ['Train', 'Test', 'Validation']
counts_pie = [train_num, test_num, val_num]
colors_pie = ['gold', 'lightcoral', 'lightskyblue']

# Create a figure with specific dimensions and subplots
fig, ax = plt.subplots(1, 2, figsize=(8, 4))  # Adjust figsize as needed

# Plotting the histogram
ax[0].bar(classes_hist, counts_hist, color=colors_hist)
ax[0].set_title('Histogram distribution')
ax[0].set_xlabel('Class')
ax[0].set_ylabel('Count')

# Plotting the pie chart
ax[1].pie(counts_pie, labels=classes_pie, colors=colors_pie, autopct='%1.1f%%', startangle=90)
ax[1].set_title('Pie Chart of training, test and validation sizes')

plt.tight_layout()  # This adjusts subplot params so that the subplots fit into the figure area.
plt.show()

print("\nThe number of training images:", train_num)
print("The number of test images:", test_num)
print("The number of validation images:", val_num)

train_num_norm, train_num_pn = counter(path_training_set, labels = True);
test_num_norm, test_num_pn= counter(path_test_set, labels = True);
val_num_norm, val_num_pn = counter(path_val_set, labels = True);

print("\nThe number of normal images: ", train_num_norm + test_num_norm + val_num_norm)
print("The number of pneumonia images: ", train_num_pn + test_num_pn + val_num_pn)

In [ ]:
# @title Visualization of two random images
# Define a transform element to resize the images as square images and transform in Tensor
tr_1 = transforms.ToTensor()                # Convert the image to a tensor

# Definition of the paths for training, test and validation set
path_training_set = "/content/chest_xray/train"
path_test_set = "/content/chest_xray/test"
path_val_set = "/content/chest_xray/val"

# Instance of classes for the Dataset
train_dataset = ChestRayDataset(path_training_set, tr_1) # Create an instance of training class
test_dataset = ChestRayDataset(path_test_set, tr_1) # Create an instance of test class
val_dataset = ChestRayDataset(path_val_set, tr_1) # Create an instance of validation class

# Generate two random numbers to select two random images from training set
random_num_1 = np.random.randint(1, len(train_dataset))
random_num_2 = np.random.randint(1, len(train_dataset))

# Take an image + label randomly
img_1, label_1 = train_dataset[random_num_1]
img_2, label_2 = train_dataset[random_num_2]

# Display the image using matplotlib library
plt.figure( figsize=(6,3))
plt.subplot(1,2,1)
plt.imshow(img_1.permute(1, 2, 0))
plt.axis('on')
plt.text(0, 0, label_1, fontsize=12, color='black', backgroundcolor='white')

plt.subplot(1,2,2)
plt.imshow(img_2.permute(1, 2, 0))
plt.axis('on')
plt.text(0, 0, label_2, fontsize=12, color='black', backgroundcolor='white')

plt.show()

print ("The shape of the first image is: ", img_1.permute(1, 2, 0).shape)
print ("The shape of the second image is: ", img_2.permute(1, 2, 0).shape)

In [ ]:
# @title Squarring

# Define a transform element to resize the images as square images and transform in Tensor
tr_1 = transforms.ToTensor()                 # Convert the image to a tensor

# Definition of the paths for training, test and validation set
path_training_set_resized = "/content/chest_xray_resized/train"
path_test_set_resized = "/content/chest_xray_resized/test"
path_val_set_resized = "/content/chest_xra_resized/val"

# Instance of classes for the Dataset
train_dataset_resized = ChestRayDataset(path_training_set_resized, tr_1) # Create an instance of training class
test_dataset_resized = ChestRayDataset(path_test_set_resized, tr_1) # Create an instance of test class
val_dataset_resized = ChestRayDataset(path_val_set_resized, tr_1) # Create an instance of validation class

# Take an image + label randomly
img_1, label_1 = train_dataset_resized[random_num_1]
img_2, label_2 = train_dataset_resized[random_num_2]

# Display the image using matplotlib library
plt.figure( figsize=(6,3))
plt.subplot(1,2,1)
plt.imshow(img_1.permute(1, 2, 0))
plt.axis('on')
plt.text(0, 0, label_1, fontsize=12, color='black', backgroundcolor='white')

plt.subplot(1,2,2)
plt.imshow(img_2.permute(1, 2, 0))
plt.axis('on')
plt.text(0, 0, label_2, fontsize=12, color='black', backgroundcolor='white')

plt.show()

print ("The shape of the first image is: ", img_1.permute(1, 2, 0).shape)
print ("The shape of the second image is: ", img_2.permute(1, 2, 0).shape)

In [ ]:
# @title Statistics after data augmentation
# Definition of the paths for training, test and validation set
path_training_set_resized_augmented = "/content/chest_xray_resized_augmented/train"

# Counting the Normal and Pneumonia images in the training set
train_norm, train_pneu = counter(path_training_set_resized_augmented, labels = True); # Count the number of normal images and pneumonia images

# Histogram Data
classes_hist = ['Normal', 'Pneumonia']
counts_hist = [train_norm, train_pneu]
colors_hist = ['aquamarine', 'blue']


plt.figure(figsize=(5, 3))
plt.bar(classes_hist, counts_hist, color=colors_hist)
plt.title('Histogram distribution')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()

print("The number of normal images:", train_norm)
print("The number of pneumonia images:", train_pneu)


In [ ]:
# @title Visualization of two sample in augmented dataset
# Instance of classes for the Dataset
train_dataset_resized_augmented = ChestRayDataset(path_training_set_resized_augmented, tr_1) # Create an instance of training class

# Generate two random numbers to select two random images from training set
random_num_1 = np.random.randint(1, len(train_dataset_resized_augmented))
random_num_2 = np.random.randint(1, len(train_dataset_resized_augmented))

# Take an image + label randomly
img_1, label_1 = train_dataset_resized_augmented[random_num_1]
img_2, label_2 = train_dataset_resized_augmented[random_num_2]

# Display the image using matplotlib library
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plt.imshow(img_1.permute(1, 2, 0))
plt.axis('on')
plt.text(0, 0, label_1, fontsize=12, color='black', backgroundcolor='white')

plt.subplot(1,2,2)
plt.imshow(img_2.permute(1, 2, 0))
plt.axis('on')
plt.text(0, 0, label_2, fontsize=12, color='black', backgroundcolor='white')

plt.show()

In [ ]:
# @title Visualization of two sample in gaussian dataset
# Instance of classes for the Dataset
path_training_set_gaussian = "/content/chest_xray_gaussian/train"
train_dataset_gaussian = ChestRayDataset(path_training_set_gaussian, tr_1) # Create an instance of training class

# Generate two random numbers to select two random images from training set
random_num_1 = np.random.randint(1, len(train_dataset_gaussian))
random_num_2 = np.random.randint(1, len(train_dataset_gaussian))

# Take an image + label randomly
img_1, label_1 = train_dataset_gaussian[random_num_1]
img_2, label_2 = train_dataset_gaussian[random_num_2]

# Display the image using matplotlib library
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plt.imshow(img_1.permute(1, 2, 0))
plt.axis('on')
plt.text(0, 0, label_1, fontsize=12, color='black', backgroundcolor='white')

plt.subplot(1,2,2)
plt.imshow(img_2.permute(1, 2, 0))
plt.axis('on')
plt.text(0, 0, label_2, fontsize=12, color='black', backgroundcolor='white')

plt.show()

In [ ]:
# @title Visualization of two sample in gaussian_he dataset
# Instance of classes for the Dataset
path_training_set_gaussian_he = "/content/chest_xray_gaussian_he/train"
train_dataset_gaussian_he = ChestRayDataset(path_training_set_gaussian_he, tr_1) # Create an instance of training class

# Generate two random numbers to select two random images from training set
random_num_1 = np.random.randint(1, len(train_dataset_gaussian_he))
random_num_2 = np.random.randint(1, len(train_dataset_gaussian_he))

# Take an image + label randomly
img_1, label_1 = train_dataset_gaussian_he[random_num_1]
img_2, label_2 = train_dataset_gaussian_he[random_num_2]

# Display the image using matplotlib library
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plt.imshow(img_1.permute(1, 2, 0))
plt.axis('on')
plt.text(0, 0, label_1, fontsize=12, color='black', backgroundcolor='white')

plt.subplot(1,2,2)
plt.imshow(img_2.permute(1, 2, 0))
plt.axis('on')
plt.text(0, 0, label_2, fontsize=12, color='black', backgroundcolor='white')

plt.show()

# Training

## Train functions

Training loop from [documentation](https://pytorch.org/tutorials/beginner/introyt/trainingyt.html)

In [ ]:
def save_metrics_to_csv(metrics_dict, name_dict):

    with open(name_dict, 'w') as f:
        json.dump(metrics_dict,f)

    print(f"{Color.GREEN}{name_dict} saved{Color.RESET}")

In [ ]:
def load_metrics_from_csv(path):

    if os.path.exists(path):
        with open(path, 'r') as f:
            loaded_data = json.load(f)

        print(f"{Color.GREEN}{path} loaded{Color.RESET}")
        return loaded_data
    else:
        print(f"{Color.RED}{path} not loaded, return {{}}{Color.RESET}")
        return {}

In [ ]:
def my_plot(train_dict, x_axis1, y_axis1, val_dict, x_axis2, y_axis2, title,
            marker1='o', linestyle1='-', color1='b',
            marker2='x', linestyle2='-', color2='r'):

    x_values1 = train_dict[x_axis1]
    y_values1 = train_dict[y_axis1]

    x_values2 = val_dict[x_axis2]
    y_values2 = val_dict[y_axis2]

    plt.figure(figsize=(8, 5))

    # Plot the first function
    plt.plot(x_values1, y_values1, marker=marker1, linestyle=linestyle1, color=color1, label="Training " + y_axis1)

    # Plot the second function
    plt.plot(x_values2, y_values2, marker=marker2, linestyle=linestyle2, color=color2, label="Validation " + y_axis2)

    plt.xlabel(f"{x_axis1}")
    plt.ylabel(f"{y_axis1}")
    plt.title(f"{title}")
    plt.legend()

    # Ensure the x-axis has integer ticks
    x_min = min(min(x_values1), min(x_values2))
    x_max = max(max(x_values1), max(x_values2))
    plt.xticks(range(x_min, x_max + 1))

    plt.show()

In [ ]:
def plot_confusion_matrix(cm, class_names, normalize=False, title='Confusion Matrix', cmap='Oranges'):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='.2f' if normalize else 'd', cmap=cmap,
                xticklabels=class_names, yticklabels=class_names)

    plt.title(title)
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

In [ ]:
def calculate_metrics(model, outputs, labels, metrics_dict, store_to_dict=True, stamp=True):

    accuracy_value = model.accuracy(outputs, labels)
    precision_value = model.precision(outputs, labels)
    recall_value = model.recall(outputs, labels)
    auroc_value = model.auroc(outputs, labels)
    f1score_value = model.f1score(outputs, labels)

    accuracy_value_item = accuracy_value.item() * 100
    precision_value_item = precision_value.item() * 100
    recall_value_item = recall_value.item() * 100
    auroc_value_item = auroc_value.item() * 100
    f1score_value_item = f1score_value.item() * 100

    metrics_current_dict = {
      'accuracy': accuracy_value_item,
      'precision': precision_value_item,
      'recall': recall_value_item,
      'auroc': auroc_value_item,
      'f1score': f1score_value_item
    }

    if store_to_dict == True:
      metrics_dict['accuracy'].append(accuracy_value_item)
      metrics_dict['precision'].append(precision_value_item)
      metrics_dict['recall'].append(recall_value_item)
      metrics_dict['auroc'].append(auroc_value_item)
      metrics_dict['f1score'].append(f1score_value_item)


    metrics_string = f"\n\taccuracy: {accuracy_value_item:.2f}, precision: {precision_value_item:.2f}, recall: {recall_value_item:.2f}, auroc: {auroc_value_item:.2f}, f1score: {f1score_value_item:.2f}"

    if stamp == True:
      print(metrics_string)
    return metrics_current_dict

In [ ]:
def train_one_epoch(model, epoch_index, training_loader, metrics_train):
    running_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting

    n_batch = len(training_loader)  # number of batches in training loader
    loss_epoch_array = []

    with tqdm(total=n_batch, desc=f'Epoch {epoch_index} ') as pbar:
        for i, data in enumerate(training_loader):

            # Every data instance is an input + label pair
            inputs, labels = data

            inputs = inputs.to(device)
            labels = labels.to(device)

            labels = labels.unsqueeze(dim=1)     # reshaping as inputs, from [0, 1] to [[0], [1]]
            labels = labels.type(torch.float32)  # preditions of network are floats

            # Zero your gradients for every batch!
            model.optimizer.zero_grad()

            # Make predictions for this batch
            outputs = model(inputs)

            # Compute the loss and its gradients
            loss = model.loss_fn(outputs, labels)
            loss_epoch_array.append(loss.item())

            loss.backward()

            # Adjust learning weights
            model.optimizer.step()

            # Gather data and report
            running_loss += loss.item()

            if i % batch_size == 0 and i>0:
                last_loss = running_loss / batch_size #1000 # loss per batch
                print(f'\r\tbatch {i+1} loss: {last_loss}', end='')
                running_loss = 0.

                del inputs, labels, outputs, loss
                gc.collect()

                if device != "cpu":
                    torch.cuda.empty_cache()

            # compute metrics in last epoch
            if i == n_batch-1:
                print("\n\tTraining metrics:", end='')
                _ = calculate_metrics(model, outputs, labels, metrics_train)

            pbar.update(1)

        # Svuota la cache CUDA dopo ogni epoca
        torch.cuda.empty_cache()

    loss_epoch_array = np.array(loss_epoch_array)
    loss_avg = np.mean(loss_epoch_array)
    return loss_avg

In [ ]:
def training(model, epochs, training_loader, validation_loader, metrics_train, metrics_validation):
    model.set_training_mode()

    best_vloss = 1000000
    current_accuracy = 0.0
    EARLY_STOPPING = 0

    for epoch in range(epochs):

        # Make sure gradient tracking is on, and do a pass over the data
        model.set_training_mode()
        avg_loss = train_one_epoch(model, epoch, training_loader, metrics_train)

        metrics_train['loss'].append(avg_loss)
        metrics_train['epoch'].append(epoch)

        running_vloss = 0.0

        model.set_eval_mode()
        validating(
            model=model,
            epoch=epoch,
            validation_loader=validation_loader,
            metrics_validation=metrics_validation
          )

        save_metrics_to_csv(metrics_train, '/content/train_metrics.json')
        save_metrics_to_csv(metrics_validation, '/content/validation_metrics.json')

        avg_vloss = metrics_validation['loss'][-1] # / ( len(validation_loader) + 1 )
        avg_loss_epoch = metrics_train['loss'][-1] # / ( len(training_loader) + 1 )
        print(f"\taverage loss epoch: {avg_loss_epoch:.4f}, average loss validation: {avg_vloss:.4f}\n")

        if avg_vloss < best_vloss:
            best_vloss = avg_loss
            model_path = model.model_name

            print(f"\t{Color.BLUE}Saving best model on validation set{Color.RESET}")
            #model.save(epoch, stamp=True)
            model.save('best', stamp=True)

        if epoch >= 1:
            loss_1 = metrics_validation['loss'][-2]
            loss_2 = metrics_validation['loss'][-1]

            if abs(loss_1 - loss_2) <= global_var['early_stop_threshold']:
                EARLY_STOPPING += 1
                print(f"{Color.RED}Early stop counter: {EARLY_STOPPING}/{global_var['early_stop_patience']}{Color.RESET}")
            else:
                EARLY_STOPPING = 0

        if EARLY_STOPPING >= global_var['early_stop_patience']:
            print(f"{Color.RED}Early Stopping{Color.RESET}")
            break

In [ ]:
def validating(model, epoch, validation_loader, metrics_validation):

  model.set_eval_mode()
  print()
  # Disable gradient computation and reduce memory consumption.
  with torch.no_grad():
      validation_metrics_array = []
      vloss_array = []
      for i, vdata in enumerate(validation_loader):

          print(f"\r\tValidation batch: {i+1}/{len(validation_loader)}", end='')

          vinputs, vlabels = vdata

          vinputs = vinputs.to(device)
          vlabels = vlabels.to(device)

          vlabels = vlabels.unsqueeze(dim=1)    # [0, 1, 0, 1] -> [[0], [1], [0], [1]]
          vlabels = vlabels.type(torch.float32)

          voutputs = model(vinputs)
          vloss = model.loss_fn(voutputs, vlabels)

          vloss_array.append(vloss.item())

          current_validation_metrics = calculate_metrics(model, voutputs, vlabels, metrics_validation, store_to_dict=False, stamp=False)
          validation_metrics_array.append(current_validation_metrics)

      #### print validation metrics
      #print(f"\r\n\tValidation")

      sums = {key: 0.0 for key in validation_metrics_array[0].keys()}

      # calculate mean values
      for metrics_dict in validation_metrics_array:
          for key in sums.keys():
              sums[key] += metrics_dict[key]

      num_dicts = len(validation_metrics_array)
      averages = {key: sums[key] / num_dicts for key in sums.keys()}

      # print metrics
      metrics_string = (
          f"\n\taccuracy: {averages['accuracy']:.2f}, "
          f"precision: {averages['precision']:.2f}, "
          f"recall: {averages['recall']:.2f}, "
          f"auroc: {averages['auroc']:.2f}, "
          f"f1score: {averages['f1score']:.2f}"
      )

      for k in averages.keys():
        metrics_validation[k].append(averages[k])

      print(metrics_string)

      metrics_validation['loss'].append( np.mean(vloss_array))
      metrics_validation['epoch'].append(epoch)

In [ ]:
def testing(model, test_loader, metrics_test):

    model.set_eval_mode()
    running_tloss = 0

    n_batch = len(test_loader)  # number of batches in training loader
    batch_size = global_var['batch_size']
    n_files = n_batch*batch_size

    with tqdm(total=n_files, desc=f'Testing ') as pbar:

        with torch.no_grad():
            for i, tdata in enumerate(test_loader):

                tinputs, tlabels = tdata

                tinputs = tinputs.to(device)
                tlabels = tlabels.to(device)

                tlabels = tlabels.unsqueeze(dim=1)    # [0, 1, 0, 1] -> [[0], [1], [0], [1]]
                tlabels = tlabels.type(torch.float32)

                if i == 0:
                    outputs_tensor = torch.empty(0, tlabels.shape[1], device=device)
                    labels_tensor = torch.empty(0, tlabels.shape[1], device=device)

                toutputs = model(tinputs)
                tloss = model.loss_fn(toutputs, tlabels)

                metrics_test['loss'].append(tloss.item())
                metrics_test['iteration'].append(i)

                running_tloss += tloss

                outputs_tensor = torch.cat((outputs_tensor, toutputs), dim=0)
                labels_tensor = torch.cat((labels_tensor, tlabels), dim=0)

                pbar.update(tlabels.shape[0])

            _ = calculate_metrics(model, outputs_tensor, labels_tensor, metrics_test)

            labels_tensor = labels_tensor.cpu()
            labels_tensor = labels_tensor.squeeze(dim=1) # [[0], [1], [0], [1]] -> [0, 1, 0, 1]
            true_labels_np = labels_tensor.numpy().astype(int)

            outputs_tensor = outputs_tensor.cpu()
            outputs_tensor = outputs_tensor.squeeze(dim=1) # [[0], [1], [0], [1]] -> [0, 1, 0, 1]
            predicted_labels_np = outputs_tensor.numpy()
            threshold = 0.5
            predicted_labels_np = (predicted_labels_np > threshold).astype(int)

    return true_labels_np, predicted_labels_np

In [ ]:
print(device)
batch_size = global_var["batch_size"]
train_epochs = global_var["train_epochs"]
tf_to_tensor = transforms.ToTensor()

# Main

In [ ]:
use_dataset_path = os.path.join(global_var['extract_directory'], global_var['dataset_name_gaussian_clahe'])
subset_dimension = global_var['subset_dim_percentage']
use_subset = global_var['use_subset']

create_subset(use_dataset_path , subset_dimension)

if use_subset == True:
  use_dataset_path = os.path.join(global_var['extract_directory'], global_var['dataset_name_gaussian_clahe']) + "_subset"

unify_folder(use_dataset_path)

full_dataset = ChestRayDataset(
    file_path= use_dataset_path,
    transform=tf_to_tensor
)

split_values = [round(len(full_dataset) * ratio) for ratio in global_var['split_value']]
train_size, test_size, val_size = split_values[0], split_values[1], split_values[2]
train_size = train_size + ( len(full_dataset) - sum(split_values) ) # adding value for due to round problems

dt_train_dataset, dt_val_dataset, dt_test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

print(f"train dt: {len(dt_train_dataset)}, samples: {train_size}")
print(f"test dt: {len(dt_test_dataset)},  samples: {test_size}")
print(f"val dt: {len(dt_val_dataset)},  samples: {val_size}")

In [ ]:
dt_train_dataloader = DataLoader(
    dt_train_dataset,
    batch_size=batch_size,
    shuffle=True
    )

dt_test_dataloader = DataLoader(
    dt_test_dataset,
    batch_size=batch_size,
    shuffle=True
    )

dt_val_dataloader = DataLoader(
    dt_val_dataset,
    batch_size=batch_size,
    shuffle=True
    )

In [ ]:
gc.collect()
if device != "cpu":
    torch.cuda.empty_cache()

jam_network = JAM(
    channels=1664,
    device=device,
    model_name=global_var['extract_directory'] + "/" + "jam_net"
    )

#jam_network.load() # Decomment this to load precedent weights
jam_network.to(device)
jam_network.set_eval_mode()

gc.collect()
if device != "cpu":
    torch.cuda.empty_cache()


train_metrics = {}
train_metrics['epoch'] = []
train_metrics['loss'] = []
train_metrics['accuracy'] = []
train_metrics['precision'] = []
train_metrics['recall'] = []
train_metrics['auroc'] = []
train_metrics['f1score'] = []


validation_metrics = {}
validation_metrics['epoch'] = []
validation_metrics['loss'] = []
validation_metrics['accuracy'] = []
validation_metrics['precision'] = []
validation_metrics['recall'] = []
validation_metrics['auroc'] = []
validation_metrics['f1score'] = []


print("You are using for training: ", device)

training(jam_network, train_epochs, dt_train_dataloader, dt_val_dataloader, train_metrics, validation_metrics)
jam_network.save('last_epoch')

save_metrics_to_csv(train_metrics, '/content/train_metrics.json')
save_metrics_to_csv(validation_metrics, '/content/validation_metrics.json')

# TEST: last epoch

In [ ]:
test_metrics = {}
test_metrics['iteration'] = []
test_metrics['loss'] = []
test_metrics['accuracy'] = []
test_metrics['precision'] = []
test_metrics['recall'] = []
test_metrics['auroc'] = []
test_metrics['f1score'] = []

jam_network.load('last_epoch')
true_labels, predicted_labels = testing(jam_network, dt_test_dataloader, test_metrics)
save_metrics_to_csv(test_metrics, '/content/test_metrics.json')


## PLOT

**LOSS**

In [ ]:
my_plot(train_metrics, 'epoch', 'loss', validation_metrics, 'epoch', 'loss', title="JAM-net Loss")

**ACCURACY**

In [ ]:
my_plot(train_metrics, 'epoch', 'accuracy', validation_metrics, 'epoch', 'accuracy', title="JAM-net Accuracy")

**CONFUSION MATRIX**

In [ ]:
class_names = ['Normal', 'Pneumonia']
cm = confusion_matrix(true_labels, predicted_labels)
plot_confusion_matrix(cm, class_names, normalize=False, title='FA-net Confusion Matrix', cmap='Oranges')

# TEST: best model

In [ ]:
test_metrics_best = {}
test_metrics_best['iteration'] = []
test_metrics_best['loss'] = []
test_metrics_best['accuracy'] = []
test_metrics_best['precision'] = []
test_metrics_best['recall'] = []
test_metrics_best['auroc'] = []
test_metrics_best['f1score'] = []

jam_network.load('best')
true_labels, predicted_labels = testing(jam_network, dt_test_dataloader, test_metrics_best)
save_metrics_to_csv(test_metrics, '/content/test_metrics_best.json')


# References
[1]  [Arshia Rehman et al.
Review on chest patogies detection systems using deep learning techniques
Artificial Intelligence Review,
2023.](https://www.researchgate.net/publication/369379105_Review_on_chest_pathogies_detection_systems_using_deep_learning_techniques)


[2] [Ziming Liu et al.
KAN: Kolmogorov-Arnold Networks
2024.](https://arxiv.org/abs/2404.19756)


[3] [Ayush Roy et al.
FA-net: A Fuzzy Attention-aided Deep Neural Network for Pneumonia Detection in Chest X-Rays
2024.](https://arxiv.org/pdf/2406.15117)

[4] [Labeled Optical Coherence Tomography (OCT) and Chest X-Ray Images for Classification](https://data.mendeley.com/datasets/rscbjbr9sj/2)

[5] [COLIN P.D. BIRCH,
A New Generalized Logistic Sigmoid Growth Equation Compared with the Richards Growth Equation,
Annals of Botany,
Volume 83, Issue 6,
1999](https://www.sciencedirect.com/science/article/pii/S0305736499908776)

